In [4]:
import numpy as np
import pandas as pd

In [5]:
# generate num of examples 

N_exp = 10000
N_con = 10000

In [6]:
click_exp = pd.Series(np.random.binomial(1,0.4,size=N_exp))
click_con = pd.Series(np.random.binomial(1,0.2,size=N_con))

In [7]:
exp_id = pd.Series(np.repeat('exp',N_exp))
con_id = pd.Series(np.repeat('con',N_con))

In [8]:
df_exp = pd.concat([click_exp,exp_id],axis=1)
df_con = pd.concat([click_con,con_id],axis=1)

In [9]:
df_exp.columns = ['click','group']
df_con.columns = ['click','group']
print (df_exp)
print(df_con)

      click group
0         1   exp
1         1   exp
2         0   exp
3         1   exp
4         1   exp
...     ...   ...
9995      1   exp
9996      0   exp
9997      1   exp
9998      1   exp
9999      1   exp

[10000 rows x 2 columns]
      click group
0         1   con
1         0   con
2         1   con
3         0   con
4         0   con
...     ...   ...
9995      0   con
9996      0   con
9997      0   con
9998      0   con
9999      1   con

[10000 rows x 2 columns]


In [10]:
df_ab_test = pd.concat([df_exp,df_con],axis =0).reset_index(drop=True)
print(df_ab_test)

       click group
0          1   exp
1          1   exp
2          0   exp
3          1   exp
4          1   exp
...      ...   ...
19995      0   con
19996      0   con
19997      0   con
19998      0   con
19999      1   con

[20000 rows x 2 columns]


In [16]:
x_con = df_ab_test.groupby('group')['click'].sum().loc['con']
x_exp = df_ab_test.groupby('group')['click'].sum().loc['exp']

In [ ]:
print(x_con,x_exp)


np.int64(2011)

In [18]:
p_con_hat = x_con/N_con
p_exp_hat = x_exp/N_exp

In [41]:
print ('probability of click control:', p_con_hat, 'probability of click experiment:', p_exp_hat)

probability of click control: 0.2011 probability of click experiment: 0.4003


In [42]:
p_pooled_hat = (x_con+x_exp)/(N_con + N_exp)
print (p_pooled_hat)

# this is when assume the two population proportions are equal. 

0.3007


In [ ]:
pooled_variance = p_pooled_hat * (1-p_pooled_hat) * (1/N_con + 1/N_exp)
SE = np.sqrt(pooled_variance)
print ('standard error is', SE)

standard error is 0.006485052197168502


In [25]:
Test_stat = (p_con_hat - p_exp_hat)/SE
print('test stat for 2 sample z test', Test_stat)

test stat for 2 sample z test -30.716792084877053


In [27]:
from scipy.stats import norm

In [ ]:
alpha = 0.05
z_crit = norm.ppf(1-alpha/2)
print ('z-critical value from standard normal distribution',z_crit)

z-critical value from standard normal distribution 1.959963984540054


In [ ]:
p_value = 2*norm.sf(abs(Test_stat))
print(round(p_value,4))

In [38]:
CI = [round((p_exp_hat - p_con_hat) - SE* z_crit,3), round((p_exp_hat-p_con_hat)+ SE*z_crit,3)]
print('Confidence interval',CI)

Confidence interval [np.float64(0.186), np.float64(0.212)]


In [ ]:
delta = 0.03. # this is practical difference 

In [ ]:
# compare the lower bond of CI to decide whether the difference is practical 